<a href="https://colab.research.google.com/github/jiin124/4th_kaggle_study/blob/main/4%EC%A3%BC%EC%B0%A8/14%E1%84%8C%E1%85%AE%E1%84%8E%E1%85%A1_%E1%84%8B%E1%85%A1%E1%86%BC%E1%84%89%E1%85%A1%E1%86%BC%E1%84%87%E1%85%B3%E1%86%AF_%E1%84%80%E1%85%AA%E1%84%87%E1%85%A1%E1%86%AB%E1%84%89%E1%85%AE_%E1%84%90%E1%85%AE%E1%84%91%E1%85%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

bank_df = pd.read_csv('UniversalBank.csv')
bank_df.head()

,ID,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,PersonalLoan,SecuritiesAccount,CDAccount,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [4]:
#학습에 사용할 특성변수 선택

X=bank_df.drop(['ID','ZIPCode','PersonalLoan'],axis=1)
y=bank_df['PersonalLoan']

In [5]:
#데이터 분할

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1,stratify=y)

In [6]:
#학습에 사용할 모델 개별 정의

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

logistic=LogisticRegression(solver='liblinear',
                            penalty='l2',
                            C=0.001,
                            random_state=1)

tree=DecisionTreeClassifier(max_depth=None,
                            criterion='entropy',
                            random_state=1)

knn=KNeighborsClassifier(n_neighbors=1,
                         p=2,
                         metric='minkowski')

https://m.blog.naver.com/gdpresent/221703566189

`solver` : default값은 liblinear, liblinearsms L1제약조건, L2 제약조건 두가지를 모두 지언하면 이것은 작은 데이터에 적합한 알고리즘임. sag,saga는 확률적경사하강법을 기반으로 하기에 대용량 데이터에 적합한 알고리즘. sag는 L1제약조건만 지원하고, saga는 L1,L2 제약조건 둘다 지원. newton-cg,lbfgs(sag,saga)이것은 멀티 클래스의 분류 모델에 씀. 현재까지는 lgfgs알고리즘이 가장 성능이 좋고 newton-cg,lbfgs도 L2 제약조건만 지원.



In [7]:
#학습에 사용할 모델 앙상블 정의

from sklearn.ensemble import VotingClassifier

voting_estimators=[('logistic',logistic),('tree',tree),('knn',knn)]
voting=VotingClassifier(estimators=voting_estimators,
                        voting='soft')

소프트보팅을 사용, 하드보팅과 소프트 보팅이 있는데 소프트보팅이 더 자주 쓰임.

보팅은 여러개의 분류기가 투표를 통해 최종 예측 결과를 결정하는 방식. 보팅 방법에는 두가지가 있는데 하드보팅은 다수결의 원칙과 비슷하다. 예측한 결과값중 다수의 분류기가 예측값을 최종 결과값으로 결정하는 것. 소프트 보팅은 분류기의 레이블 값 결정 확률의 평균값을 구한 뒤 확률이 가장 높은 레이블 값을 최종 결과값으로 결정한다. 

In [8]:
#K-fold 교차검증을 통한 모델 평가

from sklearn.model_selection import cross_val_score

clf_labels=['Logistic Regression','Decision tree','KNN','Majority voting']
all_clf=[logistic,tree,knn,voting]

for clf,label in zip(all_clf,clf_labels):
  scores=cross_val_score(estimator=clf,X=X_train,y=y_train,cv=10,scoring='roc_auc')
  print("ROC AUC %0.3f (+/- %0.3f) [%s]",(scores.mean(),scores.std(),label))

ROC AUC %0.3f (+/- %0.3f) [%s] (0.9276195033668649, 0.01984630447185705, 'Logistic Regression')
ROC AUC %0.3f (+/- %0.3f) [%s] (0.9499227861055811, 0.032735680131811405, 'Decision tree')
ROC AUC %0.3f (+/- %0.3f) [%s] (0.7120816883018249, 0.04722587272864442, 'KNN')
ROC AUC %0.3f (+/- %0.3f) [%s] (0.9724206139059355, 0.01593603549077189, 'Majority voting')


In [13]:
#GridSearch 방식을 이용한 모델 최적화

from sklearn.model_selection import GridSearchCV

params={
    'logistic__C':[0.001,0.1,100.0],
    'tree__max_depth':[1,3,5],
    'knn__n_neighbors':[1,3,5]
}

grid = GridSearchCV(estimator=voting,
                    param_grid=params,
                    cv=10,
                    scoring='roc_auc',)

grid.fit(X_train, y_train)

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.3f %r"
          % (grid.cv_results_['mean_test_score'][r], 
             grid.cv_results_['std_test_score'][r] / 2.0, 
             grid.cv_results_['params'][r]))
    
print('최적의 파라미터: %s' % grid.best_params_)
print('ACU: %.3f' % grid.best_score_)

0.934 +/- 0.009 {'knn__n_neighbors': 1, 'logistic__C': 0.001, 'tree__max_depth': 1}
0.975 +/- 0.005 {'knn__n_neighbors': 1, 'logistic__C': 0.001, 'tree__max_depth': 3}
0.978 +/- 0.005 {'knn__n_neighbors': 1, 'logistic__C': 0.001, 'tree__max_depth': 5}
0.952 +/- 0.009 {'knn__n_neighbors': 1, 'logistic__C': 0.1, 'tree__max_depth': 1}
0.981 +/- 0.005 {'knn__n_neighbors': 1, 'logistic__C': 0.1, 'tree__max_depth': 3}
0.983 +/- 0.006 {'knn__n_neighbors': 1, 'logistic__C': 0.1, 'tree__max_depth': 5}
0.955 +/- 0.009 {'knn__n_neighbors': 1, 'logistic__C': 100.0, 'tree__max_depth': 1}
0.983 +/- 0.005 {'knn__n_neighbors': 1, 'logistic__C': 100.0, 'tree__max_depth': 3}
0.985 +/- 0.005 {'knn__n_neighbors': 1, 'logistic__C': 100.0, 'tree__max_depth': 5}
0.938 +/- 0.009 {'knn__n_neighbors': 3, 'logistic__C': 0.001, 'tree__max_depth': 1}
0.981 +/- 0.005 {'knn__n_neighbors': 3, 'logistic__C': 0.001, 'tree__max_depth': 3}
0.984 +/- 0.005 {'knn__n_neighbors': 3, 'logistic__C': 0.001, 'tree__max_depth': 5